# Tweedie Regression

#### Tweedie regression is a statistical modeling technique used to analyze data that exhibit characteristics of both continuous and discrete distributions. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2019-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
dataset['Open_Close'] = (dataset['Open'] - dataset['Adj Close'])/dataset['Open']
dataset['High_Low'] = (dataset['High'] - dataset['Low'])/dataset['Low']
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,-0.005025,0.030928,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,-0.029925,0.047619,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0.002387,0.034063,0,1,0,0.012106
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0.011820,0.028986,0,0,0,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0.026190,0.044444,0,0,1,-0.021531


In [4]:
# Create Labels string
dataset['Increase_Decrease_L'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],'Increase','Decrease')
dataset['Buy_Sell_on_Open_L'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],'Buy','Sell')
dataset['Buy_Sell_L'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],'Buy','Sell')
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns,Increase_Decrease_L,Buy_Sell_on_Open_L,Buy_Sell_L
Date,,,,,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,-0.005025,0.030928,1,1,1,0.012658,Increase,Buy,Buy
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,-0.029925,0.047619,1,1,1,0.032500,Increase,Buy,Buy
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0.002387,0.034063,0,1,0,0.012106,Decrease,Buy,Sell
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0.011820,0.028986,0,0,0,0.000000,Decrease,Sell,Sell
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0.026190,0.044444,0,0,1,-0.021531,Decrease,Sell,Buy


In [5]:
X = dataset[['Open', 'High', 'Low', 'Volume']].values
y = dataset['Adj Close'].values

In [6]:
from sklearn.linear_model import TweedieRegressor

# Create and train the Tweedie regression
model = TweedieRegressor()
model.fit(X, y)

# Fit the model
model.fit(X, y)


TweedieRegressor()

In [7]:
model.coef_

array([ 1.60161029e-01,  1.64477881e-01,  1.55425866e-01, -1.87982961e-08])

In [8]:
model.intercept_

8.167780420470837

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.4, random_state=42)

In [11]:
# Create and train the Tweedie regression
model = TweedieRegressor()
model.fit(X_train, y_train)

# Fit the model
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

In [12]:
# Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.319658879925966


In [13]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
adjusted_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - X_test.shape[1] - 1)

# Print evaluation metrics
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("Coefficient of Determination:", r2)
print("Adjusted R Squared:", adjusted_r2)

MAE: 4.531842527187316
MSE: 27.075790316942157
RMSE: 5.203440238625035
Coefficient of Determination: 0.319658879925966
Adjusted R Squared: 0.3141942926161344
